In [0]:
import re
from datetime import datetime

def extract_date_from_string(input_string):
    match = re.search(
      # r'20\d{2}[-_]?(0[1-9]|1[0-2])[-_]?(0[1-9]|[12]\d|3[01])',
        r'20\d\d[-_]?(?:0[1-9]|1[0-2])[-_]?(?:0[1-9]|[12]\d|3[01])',
        input_string
    )
    if not match:
        return None
    try:
        date_str = match.group(0).replace("-", "").replace("_", "")
        return datetime.strptime(date_str, "%Y%m%d").date()
    except:
        return None

In [0]:
print(f"date from name: {extract_date_from_string('airport_full_20251231.txt')}")
print(f"date from 20251201: {extract_date_from_string('20251201')}")
print(f"date from some-file-2025-12-01morestuff.csv': {extract_date_from_string('some-file-2025-12-01morestuff.csv')}")

In [0]:
import re
from datetime import datetime

rel_path = "airport_full_20251201.txt"
match = re.search(r'20\d{2}(0[1-9]|1[0-2])(0[1-9]|[12][0-9]|3[01])', rel_path)
yyyymmdd = match.group(0) if match else None

try:
    file_date = datetime.strptime(yyyymmdd, "%Y%m%d").date() if yyyymmdd else None
except Exception:
    file_date = None
display(file_date)

In [0]:
# def airports_raw():
#   return (
#     spark.read
#       .option("header", "true")
#       .option("sep", ",")
#       .csv(file_path)
#   )

In [0]:
from pyspark.sql.functions import col

rel_path1 = "airport_full_20251201.txt"
rel_path2 = "airport_full_20251202.txt"
rel_path3 = "airport_full_20251203.txt"

df1 = spark.read.option("header", "true").option("sep", ",").csv(f"/Volumes/demos_standard/public_data/public_raw/Airport_Batches/{rel_path1}")
df2 = spark.read.option("header", "true").option("sep", ",").csv(f"/Volumes/demos_standard/public_data/public_raw/Airport_Batches/{rel_path2}")
df3 = spark.read.option("header", "true").option("sep", ",").csv(f"/Volumes/demos_standard/public_data/public_raw/Airport_Batches/{rel_path3}")

df2_ins = df2.join(
    df1,
    on="ident",
    how="left_anti"
)
df2_del = df1.join(
    df2,
    on="ident",
    how="left_anti"
)

df3_ins = df3.join(
    df2,
    on="ident",
    how="left_anti"
)
df3_del = df2.join(
    df3,
    on="ident",
    how="left_anti"
)

# Find changed rows: same ident, any field difference
join_cols = [c for c in df1.columns if c == "ident"]
other_cols = [c for c in df1.columns if c != "ident"]

df2_chg = df2.alias("d2").join(
    df1.alias("d1"),
    on="ident",
    how="inner"
).where(
    " OR ".join([f"d2.{c} != d1.{c} OR (d2.{c} IS NULL AND d1.{c} IS NOT NULL) OR (d2.{c} IS NOT NULL AND d1.{c} IS NULL)" for c in other_cols])
).select("d2.*")

df3_chg = df3.alias("d3").join(
    df2.alias("d2"),
    on="ident",
    how="inner"
).where(
    " OR ".join([f"d3.{c} != d2.{c} OR (d3.{c} IS NULL AND d2.{c} IS NOT NULL) OR (d3.{c} IS NOT NULL AND d2.{c} IS NULL)" for c in other_cols])
).select("d3.*")



print(f"df1: {df1.count()}")
print(f"df2: {df2.count()}")
print(f"df3: {df3.count()}")

print(f"df2_ins : {df2_ins.count()}")
print(f"df2_del : {df2_del.count()}")
print(f"df2_chg : {df2_chg.count()}")

# display(df2_ins)
# display(df2_del)
# display(df2_chg)

print(f"df3_ins : {df3_ins.count()}")
print(f"d3_del : {df3_del.count()}")
print(f"df3_chg : {df3_chg.count()}")